In [1]:
import os
from numpy._distributor_init import NUMPY_MKL 
from scipy.spatial import distance as dist
from imutils.video import VideoStream
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC
from threading import Thread
from imutils import face_utils
import numpy as np
import playsound
import imutils
import dlib
import cv2

In [2]:
def eye_aspect_ratio(eye):
        A = dist.euclidean(eye[1], eye[5])
        B = dist.euclidean(eye[2], eye[4])
        C = dist.euclidean(eye[0], eye[3])
        ear = (A + B) / (2.0 * C)
        return ear

In [3]:
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor('shape_predictor.dat')
(lStart, lEnd) = face_utils.FACIAL_LANDMARKS_IDXS["left_eye"]
(rStart, rEnd) = face_utils.FACIAL_LANDMARKS_IDXS["right_eye"]
(cStart, cEnd) = (6, 11)
(mStart, mEnd) = face_utils.FACIAL_LANDMARKS_IDXS["mouth"]
(fStart, fEnd) = (1, 27)

In [25]:
#Za kameru 
vs = VideoStream().start()
ret=True;
countN=0
ukZ=0
countZ=0
COUNTER=0
TOTAL=0
timer=[0,0,0]
countFrame=0

In [26]:
while True:
    
    frame=vs.read()
    
    countFrame+=1
    print("Broj frejmova: {} ", countFrame)
    
    if(countFrame%300==0):
        print("Prosao minut" )
        if(countFrame/300<3):
            print("Upisuheno: {} u {}",ukZ,  countFrame/300)
            timer[int(countFrame/600)]=ukZ
            ukZ=0
        else:
            print("Upisuheno NC: {} u {}",ukZ,  countFrame/300)
            timer[int((countFrame/300)%3)]=ukZ
            ukZ=0
            
    if(timer[0]+timer[1]+timer[2]>=5):
        cv2.putText(frame, "U poslednja 3 minuta i manje ste zevali {} puta!".format(timer[0]+timer[1]+timer[2]), (20, 200),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
        cv2.putText(frame, "Napravite pauzu", (120, 230),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
    elif( ukZ>=5):
        cv2.putText(frame, "U poslednjem minutu ste zevali {} puta!".format(ukZ), (50, 200),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
        cv2.putText(frame, "Napravite pauzu", (120, 230),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
    elif(timer[0]+timer[1]+timer[2] + ukZ>=5):
        cv2.putText(frame, "U poslednja oko 3 minuta ste zevali {} puta!".format(timer[0]+timer[1]+timer[2] + ukZ), (50, 200),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
        cv2.putText(frame, "Napravite pauzu", (120, 230),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    rects = detector(gray, 0)
        
    for rect in rects:
            shape = predictor(gray, rect)
            shape = face_utils.shape_to_np(shape)

           
            leftEye = shape[lStart:lEnd]
            rightEye = shape[rStart:rEnd]
            chin= shape[cStart:cEnd]
            mouthU=shape[50:53]
            mouthD=shape[56:59]
            leftEAR = eye_aspect_ratio(leftEye)
            rightEAR = eye_aspect_ratio(rightEye)


            
            ear = (leftEAR + rightEAR) / 2.0
            leftEyeHull = cv2.convexHull(leftEye)
            rightEyeHull = cv2.convexHull(rightEye)
            chinH=cv2.convexHull(chin)
            mouthH=cv2.convexHull(mouthU)
            mouthH2=cv2.convexHull(mouthD)
                
            cv2.drawContours(frame, [leftEyeHull], -1, (0, 255, 0), 1)
            cv2.drawContours(frame, [rightEyeHull], -1, (0, 255, 0), 1)
            cv2.drawContours(frame, [chinH], -1, (0, 255, 0), 1)
            cv2.drawContours(frame, [mouthH], -1, (0, 255, 0), 1)
            cv2.drawContours(frame, [mouthH2], -1, (0, 255, 0), 1)
                
            distance=abs(mouthU[2]-mouthD[2])

            distanceE=abs(leftEye[3]-leftEye[0])
            
            distanceC=abs(chinH[1]-0)
            if ((abs(distance[0]-distance[1]))> 35 and distanceE[0]>25) or ((abs(distance[0]-distance[1]))> 11 and distanceE[0]>11) or ((abs(distance[0]-distance[1]))>6 and distanceE[0]<12):
                    
                countZ+=1
                countN=0
            else:
                countN+=1
                if (countZ>20 and countN>3):
                    ukZ+=1
                    countZ=0
                    countN=0
                if(countN>3):
                    countZ=0
                    
            if (ear < 0.20 and distanceE[0]<30) or (ear < 0.23 and distanceE[0]>30):
                COUNTER += 1
                cv2.putText(frame, "Eye closed", (170, 30),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
                if(COUNTER/5>5):
                    cv2.putText(frame, "OPASNOST: Oci zatvorene {:.2f} sekunde".format(COUNTER/30), (200, 200),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
                    playsound.playsound('alarm.wav')
                elif (distanceC[0][1]<250 and ear < 0.20):
                    cv2.putText(frame, "OPASNOST: Brada gore", (200, 200),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
                    playsound.playsound('alarm.wav')

            else:
                if COUNTER >=2:
                    print("Eye been closed for %d", COUNTER)
                    TOTAL += 1
                    COUNTER = 0
                        
                COUNTER = 0
                cv2.putText(frame, "Eye open", (170, 30),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
            
            
            cv2.putText(frame, "Blinks: {}".format(TOTAL), (10, 30),
                cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
            cv2.putText(frame, "EAR: {:.2f}".format(ear), (300, 30),
                cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
            cv2.putText(frame, "Z: {}".format(ukZ), (100, 30),
                cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)

    cv2.imshow("Frame", frame)

    if cv2.waitKey(10) & 0xFF == ord('q'):
        break
vs.stop()
cv2.destroyAllWindows()
print("Blinks: {}",TOTAL )
print("y: {}",ukZ )

Broj frejmova: {}  1
Broj frejmova: {}  2
Broj frejmova: {}  3
Broj frejmova: {}  4
Broj frejmova: {}  5
Broj frejmova: {}  6
Broj frejmova: {}  7
Broj frejmova: {}  8
Broj frejmova: {}  9
Broj frejmova: {}  10
Broj frejmova: {}  11
Eye been closed for %d 10
Broj frejmova: {}  12
Broj frejmova: {}  13
Broj frejmova: {}  14
Broj frejmova: {}  15
Broj frejmova: {}  16
Broj frejmova: {}  17
Broj frejmova: {}  18
Broj frejmova: {}  19
Broj frejmova: {}  20
Broj frejmova: {}  21
Broj frejmova: {}  22
Broj frejmova: {}  23
Broj frejmova: {}  24
Broj frejmova: {}  25
Broj frejmova: {}  26
Broj frejmova: {}  27
Broj frejmova: {}  28
Broj frejmova: {}  29
Broj frejmova: {}  30
Broj frejmova: {}  31
Broj frejmova: {}  32
Broj frejmova: {}  33
Broj frejmova: {}  34
Broj frejmova: {}  35
Broj frejmova: {}  36
Eye been closed for %d 15
Broj frejmova: {}  37
Broj frejmova: {}  38
Broj frejmova: {}  39
Broj frejmova: {}  40
Broj frejmova: {}  41
Broj frejmova: {}  42
Broj frejmova: {}  43
Broj frejmo

Broj frejmova: {}  347
Broj frejmova: {}  348
Broj frejmova: {}  349
Broj frejmova: {}  350
Broj frejmova: {}  351
Broj frejmova: {}  352
Broj frejmova: {}  353
Broj frejmova: {}  354
Broj frejmova: {}  355
Broj frejmova: {}  356
Broj frejmova: {}  357
Broj frejmova: {}  358
Broj frejmova: {}  359
Broj frejmova: {}  360
Broj frejmova: {}  361
Broj frejmova: {}  362
Broj frejmova: {}  363
Broj frejmova: {}  364
Broj frejmova: {}  365
Eye been closed for %d 9
Broj frejmova: {}  366
Broj frejmova: {}  367
Broj frejmova: {}  368
Broj frejmova: {}  369
Broj frejmova: {}  370
Broj frejmova: {}  371
Broj frejmova: {}  372
Broj frejmova: {}  373
Broj frejmova: {}  374
Broj frejmova: {}  375
Broj frejmova: {}  376
Broj frejmova: {}  377
Broj frejmova: {}  378
Eye been closed for %d 2
Broj frejmova: {}  379
Broj frejmova: {}  380
Broj frejmova: {}  381
Broj frejmova: {}  382
Broj frejmova: {}  383
Broj frejmova: {}  384
Broj frejmova: {}  385
Broj frejmova: {}  386
Broj frejmova: {}  387
Broj fr

Broj frejmova: {}  686
Broj frejmova: {}  687
Broj frejmova: {}  688
Broj frejmova: {}  689
Broj frejmova: {}  690
Broj frejmova: {}  691
Broj frejmova: {}  692
Broj frejmova: {}  693
Broj frejmova: {}  694
Broj frejmova: {}  695
Broj frejmova: {}  696
Broj frejmova: {}  697
Broj frejmova: {}  698
Broj frejmova: {}  699
Broj frejmova: {}  700
Broj frejmova: {}  701
Broj frejmova: {}  702
Broj frejmova: {}  703
Broj frejmova: {}  704
Broj frejmova: {}  705
Eye been closed for %d 7
Broj frejmova: {}  706
Broj frejmova: {}  707
Broj frejmova: {}  708
Broj frejmova: {}  709
Broj frejmova: {}  710
Broj frejmova: {}  711
Broj frejmova: {}  712
Broj frejmova: {}  713
Broj frejmova: {}  714
Broj frejmova: {}  715
Broj frejmova: {}  716
Broj frejmova: {}  717
Broj frejmova: {}  718
Broj frejmova: {}  719
Broj frejmova: {}  720
Broj frejmova: {}  721
Broj frejmova: {}  722
Broj frejmova: {}  723
Broj frejmova: {}  724
Broj frejmova: {}  725
Broj frejmova: {}  726
Broj frejmova: {}  727
Broj frej